In [ ]:
from tf_explain.core.grad_cam import GradCAM
from tf_explain.core.smoothgrad import SmoothGrad
from keras.models import load_model

import keras
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import shap

#### **Grad-CAM visualisation**

Use the Grad-CAM XAI tool to analyse the model attention and generate visualisation heatmaps.

In [ ]:
#Load pre-trained base models
mobilenet_model = load_model("saved_model/finetuned_mobilenet.keras")
incepresnet_model = load_model("saved_model/finetuned_incepresnet.keras")
xception_model = load_model("saved_model/finetuned_xception.keras")

In [ ]:
# Select the target sample 
image = cv2.imread('test_images_directory/class_name/file_name.jpg')
image = cv2.resize(image, (224, 224))
image = image / 255.0
image = keras.utils.img_to_array(image)
data = ([image], None)
print(f"image: {image.shape}")

In [ ]:
# Use the model to predict selected sample 
def predict_categorie_img(img, model, categories):
    try:
        img = img[None, :, :, :]
    except:
        raise TypeError("test image dimension != 3")
    predict = model.predict(img)
    idx_cat = np.argmax(predict, axis=1)[0]
    return idx_cat, categories[idx_cat]

In [ ]:
# Create GradCAM explainer
explainer = GradCAM()
class_index= 1

inception_heatmap = explainer.explain(data, incepresnet_model.get_layer('inception_resnet_v2'), class_index, layer_name='conv_7b_ac')  # Change class_index as needed
mobile_heatmap = explainer.explain(data, mobilenet_model.get_layer('MobilenetV3large'), class_index, layer_name='multiply_19')  # Change class_index as needed
xception_heatmap = explainer.explain(data, xception_model.get_layer('xception'), class_index, layer_name='block14_sepconv2_act')  # Change class_index as needed

img = x_test[1000] # Specify the position numbder of the selected sample in the whole test set

# Use the base model to make predictions
inception_pred_class = predict_categorie_img(img, incepresnet_model, categories) # Use the data pre-processing file to acquire corresponding variables
mobile_pred_class = predict_categorie_img(img, mobilenet_model, categories)
xception_pred_class = predict_categorie_img(img, xception_model, categories)

# Compare the predicted label with ground truth label
true_class = y_t[1000], categories[y_t[1000]]
print(f"Xception Pred:[{xception_pred_class}]\nInceptionResNet Pred:[{inception_pred_class}]\nMobileNet Pred:[{mobile_pred_class}]\nTrue:[{true_class}]")

In [ ]:
# Plot the original image with the heatmap overlay
plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(cv2.resize(cv2.imread('test_images_directory/class_name/file_name.jpg'),(224, 224)), cv2.COLOR_BGR2RGB))
plt.title('Ground Truth Image')

plt.subplot(1, 3, 2)
plt.imshow(inception_heatmap, cmap='viridis')
plt.title('InceptionResNet HeatMap')

plt.subplot(1, 3, 3)
plt.imshow(mobile_heatmap, cmap='viridis')
plt.title('MobileNet HeatMap')
plt.show()

plt.figure(figsize=(6, 6))


plt.subplot(1, 1, 1)
plt.imshow(xception_heatmap, cmap='viridis')
plt.title('Xception HeatMap')

plt.show()

#### **Combined Grad-CAM visualisation**

Use multiple Grad-CAM heatmaps to analyse the combined model attention and generate visualisation heatmaps.

In [ ]:
# Combine multiple Grad-CAM heatmaps
def normalize_heatmap(heatmap):
    heatmap = np.maximum(heatmap, 0)
    heatmap = heatmap / np.max(heatmap)
    return heatmap

def resize_heatmap(heatmap, target_size):
    return cv2.resize(heatmap, target_size)

def combine_heatmaps(heatmaps, method='average'):
    if method == 'average':
        combined_heatmap = np.mean(heatmaps, axis=0)
    elif method == 'sum':
        combined_heatmap = np.sum(heatmaps, axis=0)
    combined_heatmap = normalize_heatmap(combined_heatmap)
    return combined_heatmap

def superimpose_heatmap_on_image(image, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), colormap)
    superimposed_image = cv2.addWeighted(heatmap, alpha, image, 1 - alpha, 0)
    return superimposed_image

# Example usage
image2 = cv2.cvtColor(cv2.resize(cv2.imread('test_images_directory/class_name/file_name.jpg'),(224, 224)), cv2.COLOR_BGR2RGB)
heatmaps2 = [inception_heatmap, mobile_heatmap, xception_heatmap]  
heatmaps2 = [normalize_heatmap(hm) for hm in heatmaps2]
heatmaps2 = [resize_heatmap(hm, (image.shape[1], image.shape[0])) for hm in heatmaps2]

combined_heatmap = combine_heatmaps(heatmaps2)
superimposed_image = superimpose_heatmap_on_image(image2, combined_heatmap)

cv2.imwrite('Combined_image.jpg', superimposed_image)

In [ ]:
plt.imshow(combined_heatmap, cmap='viridis')
plt.title('Combined HeatMap')
plt.show()

#### **Deep SHAP visualisation**

Use deep SHAP explainer to analyse the model attention and generate visualisation heatmaps.

In [ ]:
# If already load the model, no need to reload again
mobilenet_model = load_model("saved_model/finetuned_mobilenet.keras")
incepresnet_model = load_model("saved_model/finetuned_incepresnet.keras")
xception_model = load_model("saved_model/finetuned_xception.keras")

In [ ]:
test_dataset_dir = ("origin_dataset/test_images_directory")
# Create data generators
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
    test_dataset_dir,  # define the test dataset path
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

# Get one batch from each generator
test_batch = next(test_generator)

# Extract images and labels from the batches
images, targets = test_batch

In [ ]:
# Set a batch size for output sample
Batch_SIZE = 32
background_images = images[:Batch_SIZE]
background_images = background_images.astype(int)
background_targets = targets[:Batch_SIZE]
background_targets = np.argmax(background_targets, axis=1)

test_images = images[Batch_SIZE:]
test_images = test_images.astype(int)
test_targets = targets[Batch_SIZE:]
test_targets = np.argmax(test_targets, axis=1)

In [ ]:
def show_attributions(model):
    # Predict the probabilities of the digits using the test images
    output = model(test_images)
    # Get the index of the max log-probability
    pred = tf.argmax(output, axis=1)
    # Convert to numpy only once to save time
    pred_np = pred.numpy()

    # Create shap explainer
    expl = shap.DeepExplainer(model, background_images)
    
    shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.passthrough
    shap.explainers._deep.deep_tf.op_handlers["DepthwiseConv2dNative"] = shap.explainers._deep.deep_tf.passthrough
    shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough

    for i in range(0, len(test_images)):
        ti = test_images[[i]]
        sv = expl.shap_values(ti)
        shap.image_plot(sv, ti, show=False)

        # Prepare to augment the plot
        fig = plt.gcf()
        allaxes = fig.get_axes()

        # Show the actual/predicted class
        allaxes[0].set_title('Actual: {}, pred: {}'.format(
            test_targets[i], pred_np[i]))


        prob = output[i].numpy()
        for x in range(1, len(allaxes)-1):
            allaxes[x].set_title('{:.2%}'.format(prob[x-1]), fontsize=14)
        plt.show()

# Plot the attribution of chosen model
show_attributions(mobilenet_model)